In [ ]:
!pip3 install simplemma
!pip3 install Tensorflow
!pip3 install Keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 MB 8.7 MB/s eta 0:00:00


In [ ]:
import nltk
from simplemma import text_lemmatizer
import re
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV

In [ ]:
PATH='/content/stopwords.txt'
stop_words = None
with open(PATH, "r") as stop_file:
    stop_words = set(stop_file.read().splitlines())

In [ ]:
def clean_text(text):
    text = text.replace("Â", "a")
    text = text.replace("â", "a")
    text = text.replace("î", "i")
    text = text.replace("Î", "ı")
    text = text.replace("İ", "i")
    text = text.replace("I", "ı")
    text = text.replace(u"\u00A0", " ")
    text = text.replace("|", " ")

    text = re.sub(r"@[A-Za-z0-9]+", " ", text)
    text = re.sub(r"(.)\1+", r"\1\1", text)
    text = re.sub(r"https?:\/\/\S+", " ", text)
    text = re.sub(r"http?:\/\/\S+", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"#(\w+)", " ", text)
    text = re.sub(r"^\x00-\x7F]+", " ", text)
    text = re.sub(r"[^A-Za-zâîığüşöçİĞÜŞÖÇ]+", " ", text)
    text = re.sub(r"((https://[^\s]+))", " ", text)

    text = " ".join(text.lower().strip().split())
    text = text_lemmatizer(text, lang="tr")

    return " ".join([word for word in text if word not in stop_words])

In [ ]:
sentence = 'Ben bu hisse adamı batırır oldukça kötü sakın almayın aldırmayın çok pişman olursunuz şimdiden söylüyorum YTD'
sentence

'Ben bu hisse adamı batırır oldukça kötü sakın almayın aldırmayın çok pişman olursunuz şimdiden söylüyorum YTD'

In [ ]:
sentence = 'Ben bu hisse adamı batırır oldukça kötü sakın almayın aldırmayın çok pişman olursunuz şimdiden söylüyorum YTD'
new_sentence = clean_text(sentence)
#tokens = nltk.word_tokenize(new_sentence)

new_sentence

'hisse adam batır oldukça kötü sakın al al pişman şimdiden söyle ytd'

In [ ]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

yorumlar = pd.read_csv('/content/magaza_yorumlari_duygu_analizi.csv',encoding='utf-16')
yorumlar2 = pd.read_csv('/content/magaza_yorumlari.csv',encoding='utf-16')

In [ ]:
def etiketle_yorum(yorum):
    if 'Olumlu' in yorum:
        return 1
    elif 'Olumsuz' in yorum:
        return -1
    else:
        return 0

etiketle_yorum(yorumlar)
yorumlar['etiketli'] = yorumlar['Durum'].apply(etiketle_yorum)
yorumlar

etiketle_yorum(yorumlar2)
yorumlar2['etiketli'] = yorumlar2['Durum'].apply(etiketle_yorum)
yorumlar2


,Görüş,Durum,etiketli
0,Ürün gayet güzel boyutları dört kişilik aile ...,Olumlu,1
1,Ürünü kullandım gayet memnunum,Olumlu,1
2,Hızlı kargo,Olumlu,1
3,8 yıl önce lg 42 lm serisi almıştım ve tv den ...,Olumsuz,-1
4,ürün fena değil ancak gövdenin plastik olması ...,Olumlu,1
...,...,...,...
8486,Ürün güzel mavi olanı tercih ettim bana göre s...,Olumlu,1
8487,kesinlikle kullandığım en kötü rimel kirpikler...,Olumsuz,-1
8488,Fiyatı uygun gelebilir evet lakin malzeme Yani...,Olumsuz,-1
8489,indirimdeyken aldik cok memnunuz digerleriyle...,Olumlu,1


In [ ]:

import string

def clean(row):
    if isinstance(row["Görüş"], str):
        satirlar = row["Görüş"].split('\n')
        for i in range(len(satirlar)):
          satirlar[i] = clean_text(satirlar[i])

        return " ".join(satirlar).lower().translate(str.maketrans("", "", string.punctuation))
    else:
        return ""

yorumlar['clean_comment'] = yorumlar.apply(lambda row: clean(row), axis=1)
yorumlar
yorumlar2['clean_comment'] = yorumlar2.apply(lambda row: clean(row), axis=1)
yorumlar2

yorumlar = pd.concat([yorumlar,yorumlar2])
yorumlar

,Görüş,Durum,etiketli,clean_comment
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz,-1,ses kalite ergonomi rezalet sony al fiyat çin ...
1,hizli teslimat tesekkürler,Tarafsız,0,hizli teslimat tesekkür
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu,1,ses olay süper gece çalış sıkıntı yok kablo uz...
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz,-1,gel bigün kullan hemen bozoldu hiçtavsi
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz,-1,kulaklık ses kalite alet öv öv bitir yorum şüp...
...,...,...,...,...
8486,Ürün güzel mavi olanı tercih ettim bana göre s...,Olumlu,1,ürün güzel mavi tercih sence güzel firma ürün ...
8487,kesinlikle kullandığım en kötü rimel kirpikler...,Olumsuz,-1,kesinlikle kullan kötü rimel kirpik dök başla ...
8488,Fiyatı uygun gelebilir evet lakin malzeme Yani...,Olumsuz,-1,fiyat uygun gel evet lakin malzeme plastik kal...
8489,indirimdeyken aldik cok memnunuz digerleriyle...,Olumlu,1,indirim aldik memnun digerler ayni is yapiyor ...


In [ ]:
X = yorumlar.clean_comment.to_numpy()
y = yorumlar.etiketli.to_numpy()
X

array(['ses kalite ergonomi rezalet sony al fiyat çin replika ürün al iyi kesinlikle tavsiye',
       'hizli teslimat tesekkür',
       'ses olay süper gece çalış sıkıntı yok kablo uzun iş rahat çek olay son derece güzel iç rahat say',
       ...,
       'fiyat uygun gel evet lakin malzeme plastik kalite berbat çay plastik tat al kere demle akraba hediye kesinlikle al fazla ver başka marka tercih',
       'indirim aldik memnun digerler ayni is yapiyor para ver yok',
       'ürüm calismiyor kargo ilgilen hic bise anlamadi pismanlik icerisin'],
      dtype=object)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(len(X_train), len(X_test))

15936 3984


In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
model = MultinomialNB()
model.fit(X_train, y_train)
model_DT = DecisionTreeClassifier()
model_DT.fit(X_train, y_train)
clf_SVM = SVC(kernel="linear")
clf_SVM.fit(X_train, y_train)

clf_SVM.score(X_test, y_test)

clf_SVM2 = SVC(kernel="rbf")
clf_SVM2.fit(X_train, y_train)

clf_SVM2.score(X_test, y_test)
clf_best = SVC(kernel='rbf',C=100,gamma=0.001)
clf_best.fit(X_train,y_train)

SVC(C=100, gamma=0.001)

In [ ]:
clf_best = SVC(kernel='rbf',C=100,gamma=0.001)
clf_best.fit(X_train, y_train)

SVC(C=100, gamma=0.001)

In [ ]:
model.score(X_train, y_train)

0.7926079317269076

In [ ]:
model.score(X_test, y_test)

0.7700803212851406

In [ ]:
predictions_train = model.predict(X_train)
print("Train F1:", f1_score(y_train, predictions_train,average='weighted'))

predictions_test = model.predict(X_test)
print("Test F1:", f1_score(y_test, predictions_test,average='weighted'))


print("DT train accuracy:", model_DT.score(X_train, y_train))
print("DT test accuracy:", model_DT.score(X_test, y_test))

predictions_train = model_DT.predict(X_train)
print("DT Train F1:", f1_score(y_train, predictions_train,average='weighted'))

predictions_test = model_DT.predict(X_test)
print("DT Test F1:", f1_score(y_test, predictions_test,average='weighted'))



Train F1: 0.7385557243366063
Test F1: 0.7106385993023093
DT train accuracy: 0.9917796184738956
DT test accuracy: 0.8358433734939759
DT Train F1: 0.9917756441203167
DT Test F1: 0.8145941817200607


In [ ]:
print("Bayes train accuracy:", model.score(X_train, y_train))
print("Bayes test accuracy:", model.score(X_test, y_test))

predictions_train = model.predict(X_train)
print("Bayes Train F1:", f1_score(y_train, predictions_train,average='weighted'))

predictions_test = model.predict(X_test)
print("Bayes Test F1:", f1_score(y_test, predictions_test,average='weighted'))


Bayes train accuracy: 0.7926079317269076
Bayes test accuracy: 0.7700803212851406
Bayes Train F1: 0.7385557243366063
Bayes Test F1: 0.7106385993023093


In [ ]:
from sklearn.metrics import mean_squared_error
"""
print("svm train accuracy:", clf_SVM2.score(X_train, y_train))
print("svm test accuracy:", clf_SVM2.score(X_test, y_test))

predictions_train = clf_SVM2.predict(X_train)
print("svm Train F1:", f1_score(y_train, predictions_train,average='weighted'))

predictions_test = clf_SVM2.predict(X_test)
print("svm Test F1:", f1_score(y_test, predictions_test,average='weighted'))
"""

predictions_test = clf_SVM2.predict(X_test)
print("svm sq:", mean_squared_error(y_test, predictions_test))
predictions_test2 = model.predict(X_test)
print("bayes sq: ", mean_squared_error(y_test, predictions_test2))

svm sq: 0.25978915662650603
bayes sq:  0.4859437751004016


In [ ]:
vec_sen = vectorizer.transform([new_sentence])
vec_sen

<1x12663 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [ ]:
sentence = 'Bence bu hisse senedi çok kötü demeye içim elvermiyo iyi değil ama işte orta fena değil!'
#tokens = nltk.word_tokenize(new_sentence)

new_sentence = clean_text(sentence)


vec_sen = vectorizer.transform([new_sentence])

new_sentence

"""
param_grid = [
  {"C": [1, 10, 100], "kernel": ["linear"]},
  {"C": [1, 10, 100], "gamma": [0.001, 0.0001], "kernel": ["rbf"]},
 ]

grid = GridSearchCV(SVC(), param_grid, verbose = 3)

grid.fit(X_train, y_train)

print(grid.best_params_)
grid.best_estimator_.score(X_test, y_test)
"""

'\nparam_grid = [\n  {"C": [1, 10, 100], "kernel": ["linear"]},\n  {"C": [1, 10, 100], "gamma": [0.001, 0.0001], "kernel": ["rbf"]},\n ]\n\ngrid = GridSearchCV(SVC(), param_grid, verbose = 3)\n\ngrid.fit(X_train, y_train)\n\nprint(grid.best_params_)\ngrid.best_estimator_.score(X_test, y_test)\n'

In [ ]:
sentence = 'hisse sahipleri uzun vadeli mal varlıklarını sattıklarını ve geri kalan payları hissedarlar arasında eşit bir şekilde dağıtcaklarını kamu oyuna bildirdi.'
#tokens = nltk.word_tokenize(new_sentence)

new_sentence = clean_text(sentence)


vec_sen = vectorizer.transform([new_sentence])

print("cümle: ",sentence)

print("bayes ",model.predict(vec_sen))
print("d_tree ",model_DT.predict(vec_sen))
print("svm ",clf_SVM.predict(vec_sen))
print("svm_rbf ",clf_SVM2.predict(vec_sen))

cümle:  hisse sahipleri uzun vadeli mal varlıklarını sattıklarını ve geri kalan payları hissedarlar arasında eşit bir şekilde dağıtcaklarını kamu oyuna bildirdi.
bayes  [-1]
d_tree  [0]
svm  [-1]
svm_rbf  [-1]
